---
title: "Hello, Quarto"
format: html
echo: false
message: false
warning: false
---

In [ ]:
from lifelines import AalenJohansenFitter
import numpy as np
from itertools import product
import itertools
from rtichoke.helpers.sandbox_observable_helpers import *
from lifelines import CoxPHFitter
from lifelines import WeibullAFTFitter

import pandas as pd
import pickle  

with open(r'C:\Users\I\Documents\GitHub\rtichoke_python\probs_dict.pkl', 'rb') as file:
    probs_dict = pickle.load(file)

with open(r'C:\Users\I\Documents\GitHub\rtichoke_python\reals_dict.pkl', 'rb') as file:
    reals_dict = pickle.load(file)

with open(r'C:\Users\I\Documents\GitHub\rtichoke_python\times_dict.pkl', 'rb') as file:
    times_dict = pickle.load(file)


In [ ]:
fixed_time_horizons = [1, 3, 5]
stratified_by = ["probability_threshold", "ppcr"]

aj_data_combinations = create_aj_data_combinations(list(probs_dict.keys()), fixed_time_horizons, stratified_by, 0.1)

In [ ]:
# Create reference groups
reference_groups = list(probs_dict.keys())
data_to_adjust = pd.DataFrame({
  "reference_group": np.repeat(reference_groups, len(reals_dict)),
  "probs": np.concatenate([probs_dict[group] for group in reference_groups]),
  "reals": np.tile(reals_dict, len(reference_groups)),
  "times": np.tile(times_dict, len(reference_groups))
})

# # Placeholder for add_cutoff_strata function
data_to_adjust = add_cutoff_strata(data_to_adjust, by=0.1)

In [ ]:
data_to_adjust = pivot_longer_strata(data_to_adjust)

data_to_adjust["reals"] = data_to_adjust["reals"].replace({
    0: "real_negatives",
    2: "real_competing",
    1: "real_positives"
})


data_to_adjust["reals"] = pd.Categorical(data_to_adjust["reals"], categories=["real_negatives", "real_competing", "real_positives"], ordered=True)

# Splitting data by reference group
list_data_to_adjust = {k: v for k, v in data_to_adjust.groupby("reference_group")}


# # Define assumption sets
assumption_sets = [
    {"competing": "excluded", "censored": "excluded"},
    {"competing": "adjusted_as_negative", "censored": "adjusted"},
    {"competing": "adjusted_as_censored", "censored": "adjusted"},
    {"competing": "excluded", "censored": "adjusted"},
    {"competing": "adjusted_as_negative", "censored": "excluded"}
]

In [ ]:
adjusted_data_list = create_adjusted_data_list(
  list_data_to_adjust=list_data_to_adjust,
  fixed_time_horizons=fixed_time_horizons,
  assumption_sets=assumption_sets
)


In [ ]:
# Combine all adjusted data
final_adjusted_data = pd.concat(adjusted_data_list, ignore_index=True)

aj_data_combinations['strata'] = aj_data_combinations['strata'].astype(str)

final_adjusted_data['strata'] = final_adjusted_data['strata'].astype(str)

aj_data_combinations['reals'] = aj_data_combinations['reals'].astype(str)

final_adjusted_data['reals'] = final_adjusted_data['reals'].astype(str)

categories = ["real_negatives", "real_positives", "real_competing", "real_censored"]
aj_data_combinations['reals'] = pd.Categorical(aj_data_combinations['reals'], categories=categories, ordered=True)
final_adjusted_data['reals'] = pd.Categorical(final_adjusted_data['reals'], categories=categories, ordered=True)

combined_adjusted_data = aj_data_combinations.merge(final_adjusted_data, on=["reference_group", "fixed_time_horizon", "censoring_assumption", "competing_assumption", "reals", "strata"], how='left')

In [ ]:
import pandas as pd

# combined_adjusted_data = pd.read_pickle("C:\Users\I\Documents\GitHub\rtichoke_python\docs\combined_adjusted_data.pkl")

# TODO: When I read pickle something is goes wrong

ojs_define(reference_groups_data = reference_groups)

ojs_define(data = combined_adjusted_data)

```{ojs}

//| panel: input

viewof time_horizon = Inputs.range(
  [1, 5], 
  {value: 3, step: 2, label: "Time Horizon:"}
)

viewof reference_group = Inputs.radio(
  reference_groups_data, {label: "Reference Group"}, {value: 'thin'}
)

viewof stratified_by = Inputs.radio(
  ["probability_threshold", "ppcr"], {value: "probability_threshold", label: "Stratified By"}
)

viewof censored_assumption = Inputs.radio(
  ["excluded", "adjusted"], {value: "excluded", label: "Censored Assumption"}
)

viewof competing_assumption = Inputs.radio(
  ["excluded", "adjusted_as_negative", "adjusted_as_censored", "reals"], {value: "excluded", label: "Competing Assumption"}
)

```

```{ojs}

//cumulative_aj_data_filtered = transpose(cumulative_aj_data).filter(function(subset) {
//
//  return time_horizon == subset.fixed_time_horizon &&
//         censored_assumption == subset.censored_assumption &&
//         competing_assumption == subset.competing_assumption &&
//         stratified_by == subset.stratified_by && 
//         reference_group === subset.reference_group;
//})

filtered = transpose(data).filter(function(subset) {

  return time_horizon == subset.fixed_time_horizon &&
         censored_assumption == subset.censoring_assumption &&
         competing_assumption == subset.competing_assumption &&
         stratified_by === subset.stratified_by && 
         reference_group === subset.reference_group;
})

filtered


```

```{ojs}

  
Plot.plot({
  marks: [
    Plot.barY(filtered, { 
      x: "strata", 
      y: "reals_estimate", 
      fill: "reals", 
      tip: true 
    })
  ],
  color: {
    domain: ["real_positives", "real_competing", "real_negatives", "real_censored"], 
    range: ["#009e73", "#9DB4C0", "#FAC8CD", "#E3F09B"],
    legend: true
  }
})

``` 

In [ ]:
# combined_adjusted_data.dropna(subset=['reals_estimate'])
# # 

# Perform left join between aj_data_combinations and final_adjusted_data on 'strata' and 'reals_estimate'
# only when stratified_by == 'probability_threshold' for aj_data_combinations

# aj_data_combinations_prob_threshold = aj_data_combinations[aj_data_combinations['stratified_by'] == 'probability_threshold']

# # Convert 'strata' columns to strings
# aj_data_combinations_prob_threshold['strata'] = aj_data_combinations_prob_threshold['strata'].astype(str)
# final_adjusted_data['strata'] = final_adjusted_data['strata'].astype(str)

# combined_adjusted_data = aj_data_combinations_prob_threshold.merge(
#     final_adjusted_data[['strata', 'reals', 'reals_estimate']],
#     on=['strata', 'reals'],
#     how='left'
# )


# aj_data_combinations_prob_threshold[['strata']]
# final_adjusted_data[['strata']]